In [1]:
!pip install catboost

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
import catboost
from catboost import *
import sklearn
from sklearn.preprocessing import LabelEncoder
import sys

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/LG_Aimers_Phase_3/train.csv')
test = pd.read_csv('/content/drive/MyDrive/LG_Aimers_Phase_3/test.csv')
submit = pd.read_csv('/content/drive/MyDrive/LG_Aimers_Phase_3/sample_submission.csv')

In [ ]:
print(train[train['Y_Class'] == 0]['Y_Quality'].max())
print(train[train['Y_Class'] == 1]['Y_Quality'].min())
print(train[train['Y_Class'] == 1]['Y_Quality'].max())
print(train[train['Y_Class'] == 2]['Y_Quality'].min())

0.525066667
0.525085714
0.534842857
0.534950794


In [ ]:
def del_columns(train, test):
    col_list = train.columns
    nan_list = []
    nan_cnt = []
    nan_col = []
    full_list = []
    for col in col_list:
        if train[col].isnull().sum() == 0 :
            full_list.append(col)
            continue
        nan_list.append([col, train[col].isnull().sum()])
        nan_cnt.append(train[col].isnull().sum())
        nan_col.append(col)

    del_col = []
    for col in nan_list :
        if col[1] == len(train) :
            del_col.append(col[0])
    train = train.drop(columns=del_col)
    test = test.drop(columns=del_col)

    del_col = []
    col_list = train.describe().columns
    for col in col_list :
        if col == 'Y_Class':
            continue
        if col == 'Y_Quality':
            continue
        if col == 'LINE':
            continue
        if col == 'PRODUCT_CODE':
            continue
        if train[col].nunique()==1 :
            del_col.append(col)
    train = train.drop(columns=del_col)
    test = test.drop(columns=del_col)
    
    return train,test

def make_train_test_dataset(train,test):
    
    '''
    트레인데이터, 학습데이터 셋 만들기
    '''
    
    train_x = train.drop(columns=['PRODUCT_ID','PRODUCT_CODE','Y_Class','Y_Quality'])
    test_x = test.drop(columns=['PRODUCT_ID','PRODUCT_CODE'])
    train_y = train['Y_Quality']
    train_w = train[['Y_Class']]
    return train_x, test_x, train_y, train_w


def fillna(train,test,value):
    train = train.fillna(value)
    test = test.fillna(value)
    return train,test

def labelencoder(train,test,col_list):
    
    qual_col = col_list
    for i in qual_col:
        le = LabelEncoder()
        le = le.fit(train[i])
        train[i] = le.transform(train[i])

        for label in np.unique(test[i]): 
            if label not in le.classes_: 
                le.classes_ = np.append(le.classes_, label)
        test[i] = le.transform(test[i]) 
    return train,test

In [ ]:
#train, test = del_columns(train,test)
train, test = fillna(train,test,-1)
#train_x, test_x, train_y, train_w = make_train_test_dataset(train,test)
#train_x, test_x = labelencoder(train_x,test_x,['LINE'])
train['PRODUCT_CODE'] = train['PRODUCT_CODE'].astype('category')
train['LINE'] = train['LINE'].astype('category')

test['PRODUCT_CODE'] = test['PRODUCT_CODE'].astype('category')
test['LINE'] = test['LINE'].astype('category')

In [ ]:
rows_with_all_minus_one_tr = []

# 2880번째 열부터 마지막 열까지 각 행에 대해 반복
for index, row in train.iloc[:, 2880:].iterrows():
    # 행의 모든 값이 -1인지 확인
    if all(value == -1 for value in row):
        rows_with_all_minus_one_tr.append(index)

# 결과를 저장할 빈 데이터프레임 생성
result_data_tr = train.iloc[rows_with_all_minus_one_tr]



rows_with_all_minus_one_te = []

# 2880번째 열부터 마지막 열까지 각 행에 대해 반복
for index, row in test.iloc[:, 2880:].iterrows():
    # 행의 모든 값이 -1인지 확인
    if all(value == -1 for value in row):
        rows_with_all_minus_one_te.append(index)

# 결과를 저장할 빈 데이터프레임 생성
result_data_te = test.iloc[rows_with_all_minus_one_te]

train_bf = train.iloc[rows_with_all_minus_one_tr]
test_bf = test.iloc[rows_with_all_minus_one_te]
submit_bf = submit.iloc[rows_with_all_minus_one_te]

train_af= train.drop(result_data_tr.index)
test_af= test.drop(result_data_te.index)
submit_af= submit.drop(result_data_te.index)

train_bf = train_bf.reset_index(drop=True)
test_bf = test_bf.reset_index(drop=True)

train_af = train_af.reset_index(drop=True)
test_af = test_af.reset_index(drop=True)

In [ ]:
train_bf, test_bf = del_columns(train_bf,test_bf)
train_af, test_af = del_columns(train_af,test_af)

train_x_bf, test_x_bf, train_y_bf, train_w_bf = make_train_test_dataset(train_bf,test_bf)
train_x_af, test_x_af, train_y_af, train_w_af = make_train_test_dataset(train_af,test_af)

#train_x_bf, test_x_bf = labelencoder(train_x_bf,test_x_bf,['LINE'])
#train_x_af, test_x_af = labelencoder(train_x_af,test_x_af,['LINE'])

In [ ]:
len(train_x_bf.columns)

2798

In [ ]:
train_x_bf = train_x_bf.T.drop_duplicates().T
train_x_af = train_x_af.T.drop_duplicates().T

In [ ]:
len(train_x_bf.columns)

1719

In [ ]:
#1
clf = catboost.CatBoostRegressor(
    learning_rate=0.05,# 0.05
    iterations=1000, #1000
    depth=6,
    l2_leaf_reg=5, #5
    border_count=254,cat_features=['LINE'],
    random_seed=313)

clf.fit(train_x_bf, train_y_bf)
pred_bf = clf.predict(test_x_bf)
clf.fit(train_x_af, train_y_af)
pred_af = clf.predict(test_x_af)

submit_bf['Y_Class'] = pred_bf
submit_af['Y_Class'] = pred_af

In [ ]:
submit_fin = pd.concat([submit_bf,submit_af],axis=0)
submit_fin = submit_fin.sort_values('PRODUCT_ID')
submit_fin['Y_Class'][submit_fin['Y_Class'] <= 0.5250761905000001] = 0
submit_fin['Y_Class'][submit_fin['Y_Class'] >= 0.5348968255] = 2
submit_fin['Y_Class'][(submit_fin['Y_Class'] > 0.5250761905000001) & (submit_fin['Y_Class'] <= 0.5348968255)] = 1
submit_fin.to_csv('reg_drop_dup.csv',index=False)
submit_fin['Y_Class'].value_counts()

1.0    417
2.0     74
0.0     44
Name: Y_Class, dtype: int64